The following terminal-based libraries have to be installed:

1. ncbi-blast-2.12.0+ (or other versions)
https://www.ncbi.nlm.nih.gov/books/NBK569861/

2. mafft
https://mafft.cbrc.jp/alignment/software/source.html

In [2]:
#1
import os 
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# Define protein sequence database

## 1. Download your reference protein sequences database
Sequence database can be downloaded from UniProt database.
Select FASTA as the download format.
Swiss-Prot can be downloaded as a well-annotated database of protein sequences for reference:
https://www.uniprot.org/uniprotkb?facets=reviewed%3Atrue&query=%2A

To be specific to an organism, you can define the search for this organism on UniProt (e.g. E. coli)
https://www.uniprot.org/uniprotkb?facets=model_organism%3A83333&query=e.coli

## 2. Create the proper folder structure for your reference database
In the FastaDatabases folder, create a folder corresponding to your downloaded sequence scope (e.g. SwissProt, Ecoli). Transfer your downloaded FASTA file into the new folder. Afterwards, run the cell below.

In [3]:
your_sequence_scope = "SwissProt" # this should match the name of your folder, e.g. your_sequence_scope = "SwissProt"

In [4]:
db_fasta_folder = '/Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/'+your_sequence_scope+'/'
fastas = [f for f in os.listdir(db_fasta_folder) if f.endswith(".fasta")]
if len(fastas)>0:
    input_fasta = fastas[0]
else:
    print('No fasta file in selected folder! \n \
          Please, follow the steps described above to set up your reference database')
    input_fasta = ''

Split your input sequence into batches of 5000 to parallelize the sequence search later

In [5]:
# Directory for batches
batches_dir=db_fasta_folder+'/batches/'

# if batches were not created previously, split your fasta file into batches
if not os.path.exists(batches_dir):
    Path(batches_dir).mkdir(parents=True, exist_ok=True) # Create the directory or the batches
    # define number of sequences per file
    seqs_per_file = 5000
    # Make split
    smallfile = None
    with open(db_fasta_folder+input_fasta) as all_seqs_file:
        all_seqs = all_seqs_file.read()
        seqList = all_seqs.split("\n>")
        for seqno, seq in enumerate(seqList):
            if seqno % seqs_per_file == 0:
                if smallfile:
                    smallfile.close()
                small_filename = 'batch_{}.fasta'.format(seqno + seqs_per_file)
                smallfile = open(batches_dir+small_filename, "w")
            if (seq[0]=='>'):
                smallfile.write(seq)
            else:
                smallfile.write('\n>'+seq)
        if smallfile:
            smallfile.close()

# Generating profiles

## 1. Definition of functions

Use standard python requests library to send requests to UniProt for Fasta files

In [6]:
import requests

def get_url(query):
    r = requests.get(query)
    return r.text

In [7]:
def get_EC_entries(EC):
    query1="https://rest.uniprot.org/uniprotkb/search?query=ec%3A"+EC+\
            "+reviewed%3Atrue&Asort=score&columns=id%2Cec%2Cdatabase(Gene3D)%2Creviewed&format=tsv"
    return get_url(query1)

In [8]:
def get_fasta(EC):
    query="https://rest.uniprot.org/uniprotkb/search?query=ec%3A"+EC+\
           "+reviewed%3Atrue&Asort=score&columns=id%2Cec%2Cdatabase(Gene3D)%2Creviewed&format=fasta"
    return get_url(query)

In [9]:
def requestSequencesFromUniProt(rxn_id_level, EC_exactmatch, EC_Br):
    # request sequences for the promiscuous ECs
    # Example ->
    # rxn_id_level: 22750.L1
    # EC_exactmatch: 4.1.3.17
    # EC_Br: 2.1.2.11,4.1.3.B3,4.1.3.17,4.1.3.13,4.1.2.12
    
    Br_ECs = EC_Br.split(",") # [2.1.2.11,4.1.3.B3,4.1.3.17,4.1.3.13,4.1.2.12]
    if '' in Br_ECs:
        Br_ECs.remove('')
    # find original ECs (assigned ECs)
    if len(EC_exactmatch)>=1: # if any enzyme had score = 1, exact match [,4.1.3.17]
        EC_assigned = EC_exactmatch.split(",") # EC_assigned : ECs that known to catalyze this reaction, baseline
        if '' in EC_assigned:
            EC_assigned.remove('')
        EC_assigned = list(set(EC_assigned)) # remove duplicate ECs

    # create files    
    promiscuous_fasta = open(ec_fasta_folder+"sequences_"+rxn_id_level+"_promiscuous.fasta", 'w') # all sequences for all ECs
    #promiscuous_tsv = open(projectname+"/sequences_"+rxn_id_level+"_promiscuous.tsv", "w")
    exact_fasta = open(ec_fasta_folder+"sequences_"+rxn_id_level+"_exact.fasta", "w") # ECs with BridgIT score 1, 
                                                                                 # meaning original known ECs 
                                                                                 # for this reaction
    # create fasta of exact ECs
    if EC_exactmatch:
        for EC_entry in EC_assigned:
            output=get_fasta(EC_entry) # get sequence for the original EC
            Br_ECs.append(EC_entry) # To add original EC in the candidate list
            if (output!="" and output!="\n" ):
                exact_fasta.write(output+'\n')

    # Create fasta of promiscuous candidates
    Br_ECs = list(set(Br_ECs)) # remove duplicates in candidate ECs # Br_ECs is predicted by BridgIT ECs
    for BrEC in Br_ECs:
        # write FASTA file for candidates
        output = get_fasta(BrEC)
        if (output!="" and output!="\n"):
            promiscuous_fasta.write(output+"\n")

        # write tsv info for each of the sequences requested
        # Entry	Entry Name	Reviewed	Protein names	Gene Names	Organism	Length
#         output2 = get_EC_entries(org, BrEC)
#         if (output2!="" and output2!="\n"):
#             promiscuous_tsv.write(output2+'\n')

    promiscuous_fasta.close()
    exact_fasta.close()

Read the EC-based fasta files and separate them to Entry and fasta

In [10]:
def rewriteFastaFiles(rxn_id_level):
    """Rewrite the original FASTA files where sequences are split into several lines to one seq per line"""  
    input_fasta = open(ec_fasta_folder+"/sequences_"+rxn_id_level+"_promiscuous.fasta")

    # In FASTA file sequence is split into several lines, so the lines should be collected until next sequence starts
    fasta = [] # Sequence
    entry = [] # UniProt number
    seq = ""
    for line in input_fasta:
        if line[0]==">":
            if seq != "":
                fasta.append(seq) # append sequence from the previous iteration
            entry.append(line.split('|')[1]) # UniProt ID
            seq = line
        seq+="#"+line
    fasta.append(seq)

    # check if it has separated correctly
    print(len(fasta), "\t", len(entry))
    input_fasta.close()

In [11]:
def collectUniprotIdsPerLevel(file):
    """ get UniProt numbers to match them with MAFFT output to avoid duplicates """
    UniProtIds = []
    with open(file) as f:
        for line in f:
            if line[0]==">":
                UniProtIds.append(line.split('|')[1])
    return UniProtIds

In [12]:
def cleanMAFFToutputDuplciatesAndCompareWithFasta(rxn_id_level):
    file = align_folder+"BRI_"+rxn_id_level+"_promiscuous.align"
    UniProtIDs = collectUniprotIdsPerLevel(file)
    set_check = set(UniProtIDs)
    with open(file) as f, open(align_folder+"BRI_"+rxn_id_level+"_clean.align", 'w') as w:
        flag_write = False
        for line in f:
            if line[0]==">":
                if line.split('|')[1] in set_check:
                    flag_write = True
                    set_check = set_check -{line.split('|')[1]}
                else:
                    flag_write = False
            if flag_write == True:
                w.write(line)

In [13]:
def txt_replace(filename, search, replace):
    with open(filename, 'r') as f:
        filedata = f.read().replace(search,replace)
    with open(filename, 'w') as f:
        f.write(filedata)

In [14]:
def createProfile(rxn_id_level):
    
    # Create MSA of only the exact matching sequences of this reaction
    file="sequences_"+rxn_id_level+"_exact.fasta"
    create_MSA="/usr/local/bin/mafft --thread 16 --retree 1 "+ec_fasta_folder + file+\
               " > " + align_folder+"BRI_"+rxn_id_level+"_exact.align" + suppress
    #print(create_MSA)
    os.system(create_MSA)
    
    create_MSA2=" /usr/local/bin/mafft  --anysymbol  --auto "+\
               "--add "+ec_fasta_folder+"sequences_"+rxn_id_level+"_promiscuous.fasta "+\
               "--reorder "+align_folder+"BRI_"+rxn_id_level+"_exact.align  "+\
               "> "+align_folder+"BRI_"+rxn_id_level+"_promiscuous.align" + suppress
    #print(create_MSA2)
    os.system(create_MSA2)
    
    # PSSM formation is sensitive to duplicates so clean duplicates
    cleanMAFFToutputDuplciatesAndCompareWithFasta(rxn_id_level)

    create_profile="/usr/local/ncbi/blast/bin/psiblast -subject "+ec_fasta_folder+\
                   "sequences_"+rxn_id_level+"_exact.fasta"+\
                  " -in_msa "+align_folder+"BRI_"+rxn_id_level+"_clean.align  "+\
                  "-num_threads 16 -out_pssm  "+pssm_folder+"Bridgit_"+rxn_id_level+".chk" + suppress
    #print(create_profile)
    os.system(create_profile)
    
    if os.path.exists(pssm_folder+"Bridgit_"+rxn_id_level+".chk"):
        txt_replace(pssm_folder+"Bridgit_"+rxn_id_level+".chk", 'local id 1', f'local str "{rxn_id_level}"')
        txt_replace(pssm_folder+"Bridgit_"+rxn_id_level+".chk", 'accession "', f'accession "{rxn_id_level}_')
        os.remove(align_folder+"BRI_"+rxn_id_level+"_exact.align")
        os.remove(align_folder+"BRI_"+rxn_id_level+"_promiscuous.align")
        os.remove(align_folder+"BRI_"+rxn_id_level+"_clean.align")

In [15]:
def createProfileNoExactMatch(rxn_id_level):
    
    # Create MSA for the promiscuous sequences of this reaction
    file="sequences_"+rxn_id_level+"_exact.fasta"
    create_MSA="/usr/local/bin/mafft --thread 16 --retree 1 "+ec_fasta_folder + "sequences_"+rxn_id_level+"_promiscuous.fasta"+\
               " > " + align_folder+"BRI_"+rxn_id_level+"_promiscuous.align" + suppress
    #print(create_MSA)
    os.system(create_MSA)
    
    # Skip the step of adding the sequences
      
    # PSSM formation is sensitive to duplicates so clean duplicates
    cleanMAFFToutputDuplciatesAndCompareWithFasta(rxn_id_level)
    
    # Use promiscuous fasta as a subject sequence since no exact match was found
    create_profile="/usr/local/ncbi/blast/bin/psiblast -subject "+ec_fasta_folder+\
                   "sequences_"+rxn_id_level+"_promiscuous.fasta"+\
                  " -in_msa "+align_folder+"BRI_"+rxn_id_level+"_clean.align  "+\
                  "-num_threads 16 -out_pssm  "+pssm_folder+"Bridgit_"+rxn_id_level+".chk" + suppress
    print(create_profile)
    os.system(create_profile)
    
    if os.path.exists(pssm_folder+"Bridgit_"+rxn_id_level+".chk"):
        txt_replace(pssm_folder+"Bridgit_"+rxn_id_level+".chk", 'local id 1', f'local str "{rxn_id_level}"')
        txt_replace(pssm_folder+"Bridgit_"+rxn_id_level+".chk", 'accession "', f'accession "{rxn_id_level}_')
        os.remove(align_folder+"BRI_"+rxn_id_level+"_promiscuous.align")
        os.remove(align_folder+"BRI_"+rxn_id_level+"_clean.align")

## 2. Generate individual profiles per reaction

Suppress output of the command line interface

In [16]:
#suppress = " &> /dev/null"
suppress = ''

Define projectname

In [17]:
#projectname = 'biochemical_assays_hundredseven'
#projectname = 'biochemical_assays_three_promiscuous'
#projectname = 'example_orphan_reaction_annotation'
projectname = "all_known_reactions_generate_profiles"

Define paths and create corresponding folders

In [18]:
input_ec_profile = 'projects/'+projectname+"/input/input_profiles_"+projectname+".txt"

ec_fasta_folder = 'projects/'+projectname+'/output/ec_fasta/'
align_folder = 'projects/'+projectname+'/output/align/'
pssm_folder = 'projects/'+projectname+'/output/pssm/'
library_folder = 'projects/'+projectname+'/output/profile_library/'
raw_results_folder = 'projects/'+projectname+'/output/RAW_RESULTS/'
results_folder = 'projects/'+projectname+'/output/results/'

Path(ec_fasta_folder).mkdir(parents=True, exist_ok=True)
Path(align_folder).mkdir(parents=True, exist_ok=True)
Path(pssm_folder).mkdir(parents=True, exist_ok=True)
Path(library_folder).mkdir(parents=True, exist_ok=True)
Path(raw_results_folder).mkdir(parents=True, exist_ok=True)
Path(results_folder).mkdir(parents=True, exist_ok=True)

Save date and time of when the sequence search experiment was performed

In [19]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
with open('projects/'+projectname+'/logfile'+timestr+'.txt', 'w') as w:
    w.write('Project runs started at: '+ timestr)

Alternative 1 : run profile generation for several reactions based on the input .txt file

In [20]:
def getAlreadyRequestedUniptoIdsList():
    """ Identify for which reactions the fasta was already requested to not repeat the request (longest step) """
    rxn_lvls = [i.split('_')[1] for i in os.listdir(ec_fasta_folder)]
    return rxn_lvls

In [21]:
def requestAllUniprots():
    file = open(input_ec_profile)
    rxns_alreay_requested_fasta = getAlreadyRequestedUniptoIdsList()
    for line in tqdm(file.readlines()):
        # each line in file is BridgIT level, so basically for BridgIT levels 0-7
        rxn_id_level, EC_exactmatch, EC_Br = line.split("\t")[0], line.split("\t")[1], line.split("\t")[2]
        #print(rxn_id_level)
        if rxn_id_level not in rxns_alreay_requested_fasta:
            requestSequencesFromUniProt(rxn_id_level, EC_exactmatch, EC_Br)
        txt_replace(ec_fasta_folder+"sequences_"+rxn_id_level+"_promiscuous.fasta", error_message_uniprot, '')

In [22]:
error_message_uniprot = 'Error messages\nquery parameter has an invalid syntax\n'

In [23]:
# Run command to request all fasta files from UniProt
requestAllUniprots()

100%|█████████████████████████████████████| 6405/6405 [00:05<00:00, 1082.44it/s]


In [24]:
skip_ready_flag = True # Flag to skip profile generation for already ready profiles

if skip_ready_flag == True:
    reactions_ready = [i.split('_')[1].replace('.chk','') for i in os.listdir(pssm_folder)]
else:
    reactions_ready = []
    
print(len(reactions_ready))
with open(input_ec_profile) as f:
    reactions_to_make_profiles = [i.split('\t')[0] for i in f.readlines()]

reactions_to_make_profiles = set(reactions_to_make_profiles)-set(reactions_ready)
print(len(reactions_to_make_profiles)) 

def multipleProfiles():   
    file = open(input_ec_profile)
    for line in tqdm(file.readlines()):
        # each line in file is BridgIT level, so basically for BridgIT levels 0-7
        rxn_id_level, EC_exactmatch, EC_Br = line.split("\t")[0], line.split("\t")[1], line.split("\t")[2]
        if rxn_id_level in reactions_to_make_profiles:
            #print(rxn_id_level)
            EC_exactmatch = [i for i in EC_exactmatch.split(',') if not i=='']
            if os.path.exists(ec_fasta_folder+"sequences_"+rxn_id_level+"_exact.fasta"):
                #print('MATCH', rxn_id_level)
                createProfile(rxn_id_level)
            else:
                #print('NO MATCH', rxn_id_level)
                createProfileNoExactMatch(rxn_id_level)
    file.close()

5572
833


In [25]:
multipleProfiles()

  0%|                                                  | 0/6405 [00:00<?, ?it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

Check projects/all_known_reactions_generate_profiles/output/ec_fasta/sequences_1556121064L7_promiscuous.fasta
BLAST engine error: Empty CBlastQueryVector
  0%|                                          | 5/6405 [00:00<19:27,  5.48it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure whi

/usr/local/ncbi/blast/bin/psiblast -subject projects/all_known_reactions_generate_profiles/output/ec_fasta/sequences_1556363580L5_promiscuous.fasta -in_msa projects/all_known_reactions_generate_profiles/output/align/BRI_1556363580L5_clean.align  -num_threads 16 -out_pssm  projects/all_known_reactions_generate_profiles/output/pssm/Bridgit_1556363580L5.chk


  0%|                                         | 17/6405 [00:01<10:43,  9.93it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

inputfile = orig
32 x 1080 - 92 p
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.490
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing 

/usr/local/ncbi/blast/bin/psiblast -subject projects/all_known_reactions_generate_profiles/output/ec_fasta/sequences_1556631436L7_promiscuous.fasta -in_msa projects/all_known_reactions_generate_profiles/output/align/BRI_1556631436L7_clean.align  -num_threads 16 -out_pssm  projects/all_known_reactions_generate_profiles/output/pssm/Bridgit_1556631436L7.chk


  1%|▏                                        | 36/6405 [00:10<33:03,  3.21it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

BLAST query error: CAlnReader::GetSeqEntry(): Seq_entry is not available until after Read()


...

 97%|███████████████████████████████████▊ | 6192/6405 [3:36:13<02:15,  1.57it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

Check projects/all_known_reactions_generate_profiles/output/ec_fasta/sequences_2774364583L7_promiscuous.fasta
BLAST engine error: Empty CBlastQueryVector
 97%|███████████████████████████████████▊ | 6194/6405 [3:36:14<02:06,  1.67it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure whi

 97%|████████████████████████████████████ | 6236/6405 [3:36:55<02:10,  1.29it/s]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

inputfile = orig
50 x 747 - 327 p
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.490
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing 

STEP    33 / 33 
done.

Making a distance matrix from msa.. 
    0 / 34
done.

Constructing a UPGMA tree (efffree=1) ... 
   30 / 34
done.

Progressive alignment 2/2... 
STEP    33 / 33 
done.

disttbfast (aa) Version 7.490
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)

rescale = 1
dndpre (aa) Version 7.490
alg=X, model=BLOSUM62, 1.53, +0.12, -0.00, noshift, amax=0.0
0 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 2
sueff_global = 0.100000
nadd = 2
rescale = 1

   30 / 34
Segment   1/  1    1-4913
done 002-001-1  rejected..   ected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. accepted. accepted. accepted. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. accepted. accepted. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. rejected. reject

 98%|████████████████████████████████████▍| 6304/6405 [3:38:14<02:24,  1.43s/it]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

inputfile = orig
25 x 3969 - 199 p
nadd = 25
nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
    1 / 25
done.

Constructing a UPGMA tree (efffree=0) ... 
   20 / 25
done.

Progressive alignment 1/2... 
STEP    24 / 24 
done.

Making a distance matrix from msa.. 
    0 / 25
done.

Constructing a UPGM

 99%|████████████████████████████████████▌| 6325/6405 [3:39:23<04:05,  3.07s/it]nthread = 16
nthreadpair = 16
nthreadtb = 16
ppenalty_ex = 0

Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

inputfile = orig
1 x 349 - 349 p
outputhat23=16
treein = 0
compacttree = 0
minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16

Strategy:
 L-INS-i (Probably most accurate, very slow)
 Iterative refinement method (<16) with LOCAL pairwise alignment information

If unsure which option to use, try 'm

Alternative 2 : input the rxn id and ECs directly into the jupyter notebook for 1 reaction

In [ ]:
rxn_id_level = 'R11772L7'
EC_exactmatch = '2.5.1.139'
EC_Br = '2.5.1.136,2.5.1.138,2.5.1.39,2.5.1.36,2.5.1,2.5.1.70,2.5.1.80,2.5.1.139,2.5.1.111'
#requestSequencesFromUniProt(rxn_id_level, EC_exactmatch, EC_Br)
createProfile(rxn_id_level)

Finished generation of individual profile for the reaction level

## 3. Generate library of profiles 

From https://scicomp.ethz.ch/public/manual/BLAST/BLAST.pdf :

The BLAST+ package offers three categories of applications: 1.) search tools, 2.) BLAST database tools, and 3.) sequence filtering tools. The  psiblast, rpsblast, are considered search applications, as they execute a BLAST search, whereas makeblastdb is considered BLAST database applications, as they either create or examine BLAST databases.

Find more about the makeprofiledb options here: https://www.ncbi.nlm.nih.gov/books/NBK279684/table/appendices.T.makeprofiledb_application_o/

In [26]:
files_profiles = [file for file in os.listdir(pssm_folder) if file.endswith('.chk')]
with open(library_folder+'/profiles.list', 'w') as w:
    for file in files_profiles:
        w.write(pssm_folder+file+'\n')
command="makeprofiledb -in "+library_folder+\
        "profiles.list -index T -out "+library_folder+"PROFILE_EZ -title "+projectname
print(command)
os.system(command)

makeprofiledb -in projects/all_known_reactions_generate_profiles/output/profile_library/profiles.list -index T -out projects/all_known_reactions_generate_profiles/output/profile_library/PROFILE_EZ -title all_known_reactions_generate_profiles


Created protein BLAST (alias) database projects/all_known_reactions_generate_profiles/output/profile_library/PROFILE_EZ with 5573 sequences


0

## 3. Sequence annotation

For the meaning of the output columns check https://scicomp.ethz.ch/public/manual/BLAST/BLAST.pdf

In [27]:
# sequence comaprision 
import subprocess

## Parameters
num_threads=5

## Split in batches
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Run RPS-Blast in batches     
for root, dirs, files in os.walk(batches_dir, topdown=False):
    for file_names in batch(files,num_threads):
        command_batch=""
        counter=1;
        for name in file_names:
            if counter!=len(file_names):
                command_batch+='rpsblast -num_threads 16 \
                -outfmt 7 \
                -evalue 10.0 -query '+\
                os.path.join(root, name)+' -db '+library_folder+'/PROFILE_EZ -out '+\
                raw_results_folder+name[0:-6]+'.res -num_alignments 10000 -num_descriptions 10000 &\n'
            else:
                command_batch+='rpsblast -num_threads 16 \
                -outfmt 7 \
                 -evalue 10.0 -query '+\
                os.path.join(root, name)+' -db '+library_folder+'PROFILE_EZ -out '+\
                raw_results_folder+name[0:-6]+'.res -num_alignments 10000 -num_descriptions 10000 '
            counter+=1
        process = subprocess.Popen(command_batch, shell=True)
        print(command_batch+"\n\n")
        process.wait()
        print(process.returncode)

rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_460000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_460000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_135000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_135000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_55000.fasta -db projects/all_known_reacti

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_265000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_265000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_325000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_325000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_485000.fasta -db projects/all_known_rea

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_250000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_250000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_515000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_515000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_60000.fasta -db projects/all_known_reac

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_205000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_205000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_345000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_345000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_35000.fasta -db projects/all_known_reac

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_540000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_540000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_435000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_435000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_160000.fasta -db projects/all_known_rea

...

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_465000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_465000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_525000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_525000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_285000.fasta -db projects/all_known_rea

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_320000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_320000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_570000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_570000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_390000.fasta -db projects/all_known_rea

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_180000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_180000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_375000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_375000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_235000.fasta -db projects/all_known_rea

0
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_150000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_150000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_405000.fasta -db projects/all_known_reactions_generate_profiles/output/profile_library//PROFILE_EZ -out projects/all_known_reactions_generate_profiles/output/RAW_RESULTS/batch_405000.res -num_alignments 10000 -num_descriptions 10000 &
rpsblast -num_threads 16                 -outfmt 7                 -evalue 10.0 -query /Users/anastasia/Tools/BridgIT_Plus/FastaDatabases/SwissProt//batches/batch_30000.fasta -db projects/all_known_reac

0


In [28]:
## cleaning raw results

## Concat different parts of result :
os.system('cat '+raw_results_folder+'*.res > '+raw_results_folder+'merged.txt')
## Remove extra lines:
os.system("sed '/^#/d' "+raw_results_folder+"merged.txt >"+raw_results_folder+"merged_cleaned.txt")
## Add header to file
header="seq	profile	%identity	alignment length	mismatches	gap opens	q. start	q. end	s. start	s. end	evalue	bit score"
with open(raw_results_folder+"merged_cleaned.txt", 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(header.rstrip('\r\n') + '\n' + content)
        
# Sorting the results according to bit-score
df = pd.read_csv(raw_results_folder+"merged_cleaned.txt", sep='\t')
df.sort_values(by='bit score', ascending=False, inplace=True, ignore_index=True)
df_top = df.head(1000)
df.to_csv(results_folder+projectname+'_sorted_sequences_all.csv', index=False)
df_top.to_csv(results_folder+projectname+'_sorted_sequences_top_1000.csv', index=False)